In [1]:
from load_data import ArtificialData
import numpy as np
import torch
from sklearn.linear_model import LinearRegression

In [2]:
# データの読み込み
ad_train_D = ArtificialData(mode=1, N=1000, seed=0)  # 訓練データ, mode=RCT, 定数効果
ad_test_D = ArtificialData(mode=1, N=1000, seed=1)  # テストデータ, mode=RCT, 定数効果
ate_true = torch.mean(ad_test_D.mu1 - ad_test_D.mu0)

shape check has been done.
shape check has been done.


In [3]:
# 介入がRCTの場合のnaiveなYのATEの推定量と介入が共変量に影響を受ける場合のnaiveなATEの推定量の比較
ad_train_D_rct = ArtificialData(mode=0, N=1000, seed=0)  # 訓練データ, mode=RCT, 定数効果
print('rct', torch.mean(ad_train_D_rct.yf[ad_train_D_rct.t.bool()]) - torch.mean(ad_train_D_rct.yf[(1 - ad_train_D_rct.t).bool()]))
print('bias', torch.mean(ad_train_D.yf[ad_train_D.t.bool()]) - torch.mean(ad_train_D.yf[(1 - ad_train_D.t).bool()]))

shape check has been done.
rct tensor(0.3141)
bias tensor(0.8395)


In [9]:
# OLSによる因果効果の生成式を推定
X_train = ad_train_D.X
feature_train = torch.stack([X_train[:, 0], X_train[:, 1], ad_train_D.t.float()], axis=1)
reg = LinearRegression()
reg.fit(feature_train, ad_train_D.yf)

# 結果の評価
def f_treat(X, N):
    '''^f(x, 1)を算出'''
    feature_treat = torch.stack([X[:, 0], X[:, 1], torch.ones(N)], axis=1)
    return torch.from_numpy(reg.predict(feature_treat))

def f_control(X, N):
    '''^f(x, 0)を算出'''
    feature_control = torch.stack([X[:, 0], X[:, 1], torch.zeros(N)], axis=1)
    return torch.from_numpy(reg.predict(feature_control))

def loss_pehe(y_treat_hat, y_control_hat, mu1, mu0):
    effect_hat = y_treat_hat - y_control_hat  # 効果の推定量
    effect_true = mu1 - mu0  # 真の効果
    return torch.sqrt(torch.mean((effect_hat - effect_true)**2))

def loss_ate(y_treat_hat, y_control_hat, mu1, mu0):
    effect_hat = y_treat_hat - y_control_hat  # 効果の推定量
    effect_true = mu1 - mu0  # 真の効果
    ate_hat = torch.mean(effect_hat)  # ATEの推定量
    ate_true = torch.mean(effect_true)  # 真のATE
    return torch.abs(ate_hat - ate_true)

In [5]:
def evaluation(D, fname):
    # potential outcomeの推定
    y_treat_hat = f_treat(D.X, D.N)
    y_control_hat = f_control(D.X, D.N)

    # \epsilon_{PEHE}を算出
    pehe = loss_pehe(y_treat_hat, y_control_hat, D.mu1, D.mu0)

    # \epsilon_{ATE}を算出
    ate_error = loss_ate(y_treat_hat, y_control_hat, D.mu1, D.mu0)

    # 結果の表示
    print('pehe = ', pehe.item())
    print('error of ate =', ate_error.item())

    # 結果の保存
    torch.save({
        'X_test': D.X,
        'pehe': pehe,
        'ate_error': ate_error,
    }, fname)

In [10]:
# within sample(WS)での推定誤差の評価
evaluation(ad_train_D, 'results/bias_ols_WS.pt')

pehe =  1.5380135209852597e-07
error of ate = 2.682209014892578e-07


In [11]:
# out of sample(OoS)での推定誤差を評価
evaluation(ad_test_D, 'results/bias_ols_OoS.pt')

pehe =  1.552297703710792e-07
error of ate = 2.682209014892578e-07
